In [1]:
#Set environment options
import pandas as pd
# settings to display all columns
pd.set_option("display.max_columns", None)

### Set environmental variables and install libraries

# Call nba_api for all NBA games in a given season

In [2]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams
import time

#get nba team data including id's
nba_teams = pd.DataFrame(teams.get_teams())
#Create list of NBA team id's
team_ids = nba_teams['id'].tolist()

#season data should be pulled for 
s = '2022-23'


#function to call api and get all NBA games for a given season
def get_all_games(team__ids,season):
    team_season = []
    for i in team_ids:
        games = leaguegamefinder.LeagueGameFinder(team_id_nullable = i,season_nullable = season,season_type_nullable ='Regular Season').get_data_frames()[0]
        team_season.append(games)
        nba_season = pd.concat(team_season)
        time.sleep(1)
    return(nba_season)

nba_season = get_all_games(team_ids,s)

In [3]:
nba_season.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 0 to 12
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SEASON_ID          350 non-null    object 
 1   TEAM_ID            350 non-null    int64  
 2   TEAM_ABBREVIATION  350 non-null    object 
 3   TEAM_NAME          350 non-null    object 
 4   GAME_ID            350 non-null    object 
 5   GAME_DATE          350 non-null    object 
 6   MATCHUP            350 non-null    object 
 7   WL                 343 non-null    object 
 8   MIN                350 non-null    int64  
 9   PTS                350 non-null    int64  
 10  FGM                350 non-null    int64  
 11  FGA                350 non-null    int64  
 12  FG_PCT             350 non-null    float64
 13  FG3M               350 non-null    int64  
 14  FG3A               350 non-null    int64  
 15  FG3_PCT            350 non-null    float64
 16  FTM                350 non-

Get team stats for specific team

In [14]:
# subset TEAM_ABBREVIATION = 'DAL'
NBA_seasons_DAL = nba_season[nba_season.MATCHUP.str.contains('DAL',regex=False)]

In [4]:
#function to combine matchup data
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        #pass
        return None
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result
    

In [8]:
# Combine the game rows into one. By default, the home team will be TEAM_A.
game_df = combine_team_games(nba_season,keep_method='home')
game_df.head()

,SEASON_ID,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,GAME_ID,GAME_DATE,MATCHUP_A,WL_A,MIN_A,PTS_A,FGM_A,FGA_A,FG_PCT_A,FG3M_A,FG3A_A,FG3_PCT_A,FTM_A,FTA_A,FT_PCT_A,OREB_A,DREB_A,REB_A,AST_A,STL_A,BLK_A,TOV_A,PF_A,PLUS_MINUS_A,TEAM_ID_B,TEAM_ABBREVIATION_B,TEAM_NAME_B,MATCHUP_B,WL_B,MIN_B,PTS_B,FGM_B,FGA_B,FG_PCT_B,FG3M_B,FG3A_B,FG3_PCT_B,FTM_B,FTA_B,FT_PCT_B,OREB_B,DREB_B,REB_B,AST_B,STL_B,BLK_B,TOV_B,PF_B,PLUS_MINUS_B
1,22022,1610612737,ATL,Atlanta Hawks,0022200173,2022-11-10,ATL vs. PHI,None,169,65,22,59,0.373,3,18,0.167,18,20,0.900,6,26,32,10,5,0,7,11,6.0,1610612755,PHI,Philadelphia 76ers,PHI @ ATL,None,155,60,23,55,0.418,3,17,0.176,11,13,0.846,5,29,34,10,1,4,11,16,-5.0
5,22022,1610612737,ATL,Atlanta Hawks,0022200162,2022-11-09,ATL vs. UTA,L,240,119,51,109,0.468,8,28,0.286,9,11,0.818,17,37,54,24,5,5,11,16,-6.0,1610612762,UTA,Utah Jazz,UTA @ ATL,W,239,125,44,94,0.468,17,39,0.436,20,24,0.833,13,34,47,27,5,6,13,20,6.0
9,22022,1610612737,ATL,Atlanta Hawks,0022200149,2022-11-07,ATL vs. MIL,W,242,117,46,99,0.465,11,29,0.379,14,17,0.824,12,36,48,23,11,3,12,18,19.0,1610612749,MIL,Milwaukee Bucks,MIL @ ATL,L,241,98,36,87,0.414,13,40,0.325,13,16,0.813,6,35,41,21,5,9,18,18,-19.0
13,22022,1610612737,ATL,Atlanta Hawks,0022200134,2022-11-05,ATL vs. NOP,W,265,124,47,103,0.456,9,36,0.250,21,25,0.840,12,38,50,28,12,8,13,23,3.0,1610612740,NOP,New Orleans Pelicans,NOP @ ATL,L,264,121,47,108,0.435,7,23,0.304,20,28,0.714,19,40,59,23,8,6,16,24,-3.0
18,22022,1610612752,NYK,New York Knicks,0022200110,2022-11-02,NYK vs. ATL,L,240,99,37,90,0.411,10,36,0.278,15,20,0.750,12,40,52,23,3,9,16,19,-13.0,1610612737,ATL,Atlanta Hawks,ATL @ NYK,W,239,112,44,107,0.411,12,34,0.353,12,16,0.750,17,34,51,28,11,4,7,18,13.0


In [10]:
from nba_api.stats.endpoints import gamerotation

gr = gamerotation.GameRotation(game_id= '0022200149')

gamerotation_away = gr.get_data_frames()[0]
gamerotation_home = gr.get_data_frames()[1]